In [1]:
# task4
# Впишите ваш логин. Например "vpetrov"
user = "t.starzhevskij"

# Дз было делать очень неудобно

## Особенно мне очень понравилось что кластер, в один из моментов своего ресета, решил стереть мой ноутбук, который я делал ночами. Мне пришлось делать его заново, я очень злился...

## Надеюсь в дальнейшем со Spark мне никогда не придется работать

In [2]:
# В данной ячейке параметры не меняем! 
from pyspark.sql.types import *
import pyspark.sql.functions as sf

current_dt = "2023-03-01"

demography_path = "/user/{}/data/data3/ok/coreDemography".format(user)
country_path = "/user/{}/data/data3/ok/geography/countries.csv".format(user)
rs_city_path = "/user/{}/data/data3/ok/geography/rs_city.csv".format(user)

# Путь до результата
output_path = "/user/{}/task4".format(user)

In [3]:
# Ваш код пишите ниже. Ячейки можно добавлять

In [4]:
city_path = "/user/{}/data/data3/rosstat/city.csv".format(user)
product_path = "/user/{}/data/data3/rosstat/product.csv".format(user)
prices_path = "/user/{}/data/data3/rosstat/price/*".format(user)
products_for_stat_path = "/user/{}/data/data3/rosstat/products_for_stat.csv".format(user)

In [32]:
import os
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

os.environ["HADOOP_CONF_DIR"]="/etc/hadoop/conf"
os.environ["HIVE_HOME"]="/usr/lib/hive"
os.environ["METASTORE_PORT"]="9083"
# os.environ["SPARK_HOME"]="/usr/hdp/current/spark2-client"
# os.environ["JAVA_HOME"]="/usr/java/jdk1.8.0_191/jre"

spark = pyspark.sql.SparkSession.builder\
    .master("yarn")\
    .appName("t_starzhevsky")\
    .config("spark.executor.instances", "1")\
    .config("spark.executor.memory", "1G")\
    .config("spark.executor.cores", "2")\
    .config("spark.dynamicAllocation.enabled", "false")\
    .config("spark.dynamicAllocation.executorIdleTimeout", "300s")\
    .config("spark.dynamicAllocation.maxExecutors", "1000")\
    .config("spark.driver.memory", "1G")\
    .config("spark.driver.maxResultSize", "1G")\
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")\
    .config("spark.kryoserializer.buffer.max", "1024m")\
    .enableHiveSupport()\
    .getOrCreate()

spark_context = spark.sparkContext

24/04/20 18:30:36 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [6]:
print("""
Urls:
Yarn       http://91.219.226.252:8088/cluster/scheduler
Spark      http://91.219.226.252:8088/proxy/{app_id}/stages/
App info   http://91.219.226.252:8088/cluster/app/{app_id}/
""".format(app_id=spark_context.applicationId))


Urls:
Yarn       http://91.219.226.252:8088/cluster/scheduler
Spark      http://91.219.226.252:8088/proxy/application_1713527372500_0171/stages/
App info   http://91.219.226.252:8088/cluster/app/application_1713527372500_0171/



In [7]:
!hdfs dfs -ls /user/t.starzhevskij/data/data3

Found 3 items
-rw-r--r--   3 t.starzhevskij hadoop       6148 2024-04-14 20:09 /user/t.starzhevskij/data/data3/.DS_Store
drwxr-xr-x   - t.starzhevskij hadoop          0 2024-04-14 20:09 /user/t.starzhevskij/data/data3/ok
drwxr-xr-x   - t.starzhevskij hadoop          0 2024-04-14 20:09 /user/t.starzhevskij/data/data3/rosstat


In [8]:
!hdfs dfs -cat /user/t.starzhevskij/data/data3/ok/info.txt

Описание полей:
id - id пользователя
create_date - дата создания профиля
birth_date - дата рождения (unixtime / (24*60*60))
gender - пол (2 - женщина, 1 - мужчина)
id_country - id страны
id_location - id местоположения
login_region - номер региона

# Задание

## Необходимо собрать статистику для товаров из products_for_stat.csv (и только для товаров из этого файла!).
> Все результаты сохранять в hdfs

### 1. Для предложенных товаров необходимо вычислить минимальную, максимальную и среднюю цену по всем городам.
#### Файл price_stat
- product_id
- min_price
- max_price
- price_avg

### 2. Из набора данных ОК следует для всех пользователей из городов, цена на товары в которых выше средней, собрать статистику, которая будет содержать:
#### Файл ok_dem
- city_name название города
- user_cnt число пользователей из этого города
- age_avg средний возраст пользователей
- men_cnt число пользователей мужчин
- women_cnt число пользователей женщин
- men_share доля мужчин
- women_share доля женщин (датасет ok_dem)

### 3. Из полученного датасета нужно будет выбрать города с
- максимальным и минимальным средним возрастом
- максимальной долей мужчин
- максимальной долей женщин

### 4. Для этих городов в данных Росстат нужно будет выбрать:
#### Файл product_stat
- city_name
- cheapest_product_name самый дешевый товар
- most_expensive_product_name самый дорогой товар
- price_difference разницу в цене между самым дорогим и самым дешевым

### Требования к решению:
- При желании можно реализовать шаги на MapReduce, но как минимум один из шагов должен быть раелизован на Spark
- При вычислении возраста пользователей будем вычислять его для фиксированной даты current_dt. Параметр current_dt будем считать равным {current_dt}
- Разделитель в итоговых файлах с данными - точка с запятой (;)
- Все вещественные числа должны быть округлены до 2 знаков после запятой.
- Результаты пишем в директорию на hdfs {output_path}. Если вам потребуется сохранять промежуточные результаты или предподготовленные наборы данных предлагается сохранять их в поддиректории {output_path}/stage с осмысленным названием.


### Описание сырых данных (rosstat в data3.zip):
1. city.csv - справочник городов (city[string], city_id[int])
2. product.csv - справочник товаров и услуг (product[string], product_id[int])
3. /price - данные по ценам на товары в разных городах (city_id[int], product_id[int], price[float])
4. products_for_stat.csv - идентификаторы товаров для которых нужно собрать статистику (product_id[int]).

### Описание сырых данных (ok в hadoop3.zip).
1. coreDemography - данные демографии пользователей (user_id, create_date, birth_date (число дней с 1970-01-01), gender, id_country, id_city, login_region)
2. Справочник стран geography/countries.csv (name,id)
3. Справочник соответствия городов ОК - Росстат geography/rs_city.csv (ok_city_id, rs_city_id)


# Решение

### 1. Для предложенных товаров необходимо вычислить минимальную, максимальную и среднюю цену по всем городам.
#### Файл price_stat
- product_id
- min_price
- max_price
- price_avg

In [9]:
from pyspark.sql.types import *
import pyspark.sql.functions as sf

In [10]:
from pyspark.sql import types

In [40]:
cities_df = (
    spark.read
        .option("header", "false")
        .option("sep", ";")
        .csv(city_path)
)

products_df = (
    spark.read
        .option("header", "false")
        .option("sep", ";")
        .csv(product_path)
)

prices_df = (
    spark.read
        .option("header", "false")
        .option("sep", ";")
        .csv(prices_path)
)

products_for_stat_df = (
    spark.read
        .option("header", "false")
        .option("sep", ";")
        .csv(products_for_stat_path)
)

In [12]:
def show_schema(df):
    df.printSchema()
    print(f"Total rows: {df.count()}")
    df.show(3)

In [41]:
show_schema(cities_df)
show_schema(products_df)
show_schema(prices_df)
show_schema(products_for_stat_df)

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)

Total rows: 284
+------------+---+
|         _c0|_c1|
+------------+---+
|    Белгород|  1|
|      Губкин|  2|
|Старый Оскол|  3|
+------------+---+
only showing top 3 rows

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)

Total rows: 514
+--------------------+---+
|                 _c0|_c1|
+--------------------+---+
|     Фарш мясной, кг|  1|
|Пельмени, манты, ...|  2|
|  Печень говяжья, кг|  3|
+--------------------+---+
only showing top 3 rows

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)

Total rows: 145976
+---+---+------+
|_c0|_c1|   _c2|
+---+---+------+
|193|437|529,38|
|126| 34|684,46|
|274|416|  NULL|
+---+---+------+
only showing top 3 rows

root
 |-- _c0: string (nullable = true)

Total rows: 464
+---+
|_c0|
+---+
|355|
|446|
|114|
+---+
only showing top 3 rows



In [42]:
# Приводим типы и задаем названия полей.
cities_df = (
    cities_df
    .select(
        sf.col("_c0").alias("city"),
        sf.col("_c1").cast(IntegerType()).alias("city_id")
    )
)

products_df = (
    products_df
    .select(
        sf.col("_c0").alias("product"),
        sf.col("_c1").cast(IntegerType()).alias("product_id")
    )
)

prices_df = (
    prices_df
    .select(
        sf.col("_c0").cast(IntegerType()).alias("city_id"),
        sf.col("_c1").cast(IntegerType()).alias("product_id"),
        # sf.replace(sf.col("_c2"), ",", ".").cast(DoubleType()).alias("price")
        sf.regexp_replace(
                    sf.col("_c2"), 
                    ",",
                    ".").cast(types.FloatType()).alias("price")
    )
)

products_for_stat_df = (
    products_for_stat_df
    .select(
        sf.col("_c0").cast(IntegerType()).alias("product_id")
    )
)

In [43]:
show_schema(cities_df)
show_schema(products_df)
show_schema(prices_df)
show_schema(products_for_stat_df)

root
 |-- city: string (nullable = true)
 |-- city_id: integer (nullable = true)

Total rows: 284
+------------+-------+
|        city|city_id|
+------------+-------+
|    Белгород|      1|
|      Губкин|      2|
|Старый Оскол|      3|
+------------+-------+
only showing top 3 rows

root
 |-- product: string (nullable = true)
 |-- product_id: integer (nullable = true)

Total rows: 514
+--------------------+----------+
|             product|product_id|
+--------------------+----------+
|     Фарш мясной, кг|         1|
|Пельмени, манты, ...|         2|
|  Печень говяжья, кг|         3|
+--------------------+----------+
only showing top 3 rows

root
 |-- city_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- price: float (nullable = true)

Total rows: 145976
+-------+----------+------+
|city_id|product_id| price|
+-------+----------+------+
|    193|       437|529.38|
|    126|        34|684.46|
|    274|       416|  NULL|
+-------+----------+------+
only show

In [44]:
# Статистика цен на товары по городам
price_stat_df = (
    products_for_stat_df
    .join(prices_df, "product_id", how='inner')
    .groupBy(prices_df.product_id)
    .agg(
        sf.round(sf.max(prices_df.price), 2).alias("max_price"),
        sf.round(sf.min(prices_df.price), 2).alias("min_price"),
        sf.round(sf.avg(prices_df.price), 2).alias("avg_price")
    )
    .orderBy(sf.col("max_price").desc())
)

In [45]:
show_schema(price_stat_df)
price_stat_df.count()

root
 |-- product_id: integer (nullable = true)
 |-- max_price: float (nullable = true)
 |-- min_price: float (nullable = true)
 |-- avg_price: double (nullable = true)

Total rows: 464


+----------+---------+---------+----------+
|product_id|max_price|min_price| avg_price|
+----------+---------+---------+----------+
|       323|1661591.2|595015.94|1008423.41|
|       324|1319373.0| 364593.6| 623588.98|
|       322| 750167.4|468359.28| 556312.04|
+----------+---------+---------+----------+
only showing top 3 rows



464

### 2. Из набора данных ОК следует для всех пользователей из городов, цена на товары в которых выше средней, собрать статистику, которая будет содержать:
#### Файл ok_dem
- city_name название города
- user_cnt число пользователей из этого города
- age_avg средний возраст пользователей
- men_cnt число пользователей мужчин
- women_cnt число пользователей женщин
- men_share доля мужчин
- women_share доля женщин (датасет ok_dem)

In [50]:
# (user_id, create_date, birth_date (число дней с 1970-01-01), gender, id_country, id_city, login_region)
users_df = (
    spark.read
        .option("header", "false")
        .option("sep", "\t")
        .csv(demography_path)
)

# Справочник стран geography/countries.csv (name,id)
countries_df = (
    spark.read
        .option("header", "false")
        .option("sep", ",")
        .csv(country_path)
)

rs_city_df = (
    spark.read
        .option("header", "false")
        .option("sep", "\t")
        .csv(rs_city_path)
)

rs_city_df = (
    rs_city_df
    .select(
        sf.col("_c0").cast(types.IntegerType()).alias("ok_city_id"),
        sf.col("_c1").cast(types.IntegerType()).alias("rs_city_id"),
    )
)

users_df = (
    users_df
    .select(
        sf.col("_c0").cast(types.IntegerType()).alias("user_id"),
        sf.col("_c1").cast(types.LongType()).alias("create_date"),
        sf.from_unixtime(sf.col("_c2") * (24 * 60 * 60)).alias('birth_date').cast(types.DateType()),
        sf.col("_c2").cast(types.LongType()).alias("birth_date_days"),
        sf.col("_c3").cast(types.ShortType()).alias("gender"),
        sf.col("_c4").cast(types.LongType()).alias("country_id"),
        sf.col("_c5").cast(types.IntegerType()).alias("city_id"),
        sf.col("_c6").cast(types.IntegerType()).alias("login_region")
    )
)

In [51]:
users_df.orderBy(sf.col('birth_date').desc()).show(3)
show_schema(users_df)
show_schema(rs_city_df)
show_schema(countries_df)

+--------+-------------+----------+---------------+------+-----------+-------+------------+
| user_id|  create_date|birth_date|birth_date_days|gender| country_id|city_id|login_region|
+--------+-------------+----------+---------------+------+-----------+-------+------------+
|12535722|1347135521527|2010-12-31|          14974|     1|10414533690|2215192|          27|
|15439554|1451145302590|2010-12-26|          14969|     2|10414533690|1078547|           2|
| 3322377|1274165239477|2010-12-24|          14967|     2|10414533690|1078547|          69|
+--------+-------------+----------+---------------+------+-----------+-------+------------+
only showing top 3 rows

root
 |-- user_id: integer (nullable = true)
 |-- create_date: long (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- birth_date_days: long (nullable = true)
 |-- gender: short (nullable = true)
 |-- country_id: long (nullable = true)
 |-- city_id: integer (nullable = true)
 |-- login_region: integer (nullable = true

In [48]:
# Города, цена на товары в которых выше средней
# По факту оказалось не нужным, но оставлю
# citites_for_stat_df = (
#     prices_df
    
#     .join(price_stat_df, "product_id")
#     .select(
#         prices_df.city_id,
#         prices_df.product_id,
#         prices_df.price,
#         price_stat_df.avg_price
#     )
#     .where(prices_df.price > price_stat_df.avg_price)
#     .orderBy(prices_df.price.desc())
# )

# Города с подсчетом количества продуктов, цены на которые выше средних
citites_for_stat_df = (
    prices_df
    .join(price_stat_df, "product_id")
    .where(prices_df.price > price_stat_df.avg_price)
    .groupBy(prices_df.city_id)
    .agg(
        sf.count(prices_df.city_id).alias("product_cnt")
    )
    .orderBy(sf.col("product_cnt").desc())
    .join(rs_city_df, rs_city_df.rs_city_id == prices_df.city_id)
    .select(
        sf.col("ok_city_id"),
        sf.col("rs_city_id"),
        sf.col("product_cnt")
    )
)

show_schema(citites_for_stat_df)

root
 |-- ok_city_id: integer (nullable = true)
 |-- rs_city_id: integer (nullable = true)
 |-- product_cnt: long (nullable = false)



Total rows: 284


+----------+----------+-----------+
|ok_city_id|rs_city_id|product_cnt|
+----------+----------+-----------+
|    867411|       148|         77|
|   1078547|       243|         49|
|   1531654|        31|        242|
+----------+----------+-----------+
only showing top 3 rows



In [49]:
# city_name название города
# user_cnt число пользователей из этого города
# age_avg средний возраст пользователей

# men_cnt число пользователей мужчин
# women_cnt число пользователей женщин
# men_share доля мужчин
# women_share доля женщин (датасет ok_dem)

current_date = sf.lit(current_dt).cast(types.DateType())
age = sf.floor(sf.datediff(current_date, users_df.birth_date) / 365.25).alias('age')

ok_dem_df = (
    users_df
    .join(citites_for_stat_df, citites_for_stat_df.ok_city_id == users_df.city_id)
    .select(
        citites_for_stat_df.ok_city_id,
        citites_for_stat_df.rs_city_id,
        users_df.user_id,
        users_df.gender,
        age,
    )
    
    .groupBy(citites_for_stat_df.ok_city_id, citites_for_stat_df.rs_city_id)
    .agg(
        sf.count(users_df.user_id).alias("user_cnt"),
        sf.floor(sf.avg(sf.col("age"))).alias("age_avg"),
        sf.sum(sf.when(users_df.gender == 2, 1).otherwise(0)).alias("men_cnt"),
        sf.sum(sf.when(users_df.gender == 1, 1).otherwise(0)).alias("women_cnt")
    )
    .withColumn("men_share", sf.round(sf.col("men_cnt") / sf.col('user_cnt'), 2))
    .withColumn("women_share", sf.round(sf.col("women_cnt") / sf.col('user_cnt'), 2))
)

ok_dem_df.show()
show_schema(ok_dem_df)

+----------+----------+--------+-------+-------+---------+---------+-----------+
|ok_city_id|rs_city_id|user_cnt|age_avg|men_cnt|women_cnt|men_share|women_share|
+----------+----------+--------+-------+-------+---------+---------+-----------+
|    867411|       148|     177|     39|    144|       33|     0.81|       0.19|
|   1078547|       243|   34829|     40|  26653|     8176|     0.77|       0.23|
|   1531654|        31|      21|     46|     16|        5|     0.76|       0.24|
|   2368818|       251|      29|     43|     24|        5|     0.83|       0.17|
|    241372|        85|      26|     43|     18|        8|     0.69|       0.31|
|  12141837|       137|      61|     34|     47|       14|     0.77|       0.23|
|    684283|        65|      36|     41|     26|       10|     0.72|       0.28|
|   2568145|       255|      48|     47|     37|       11|     0.77|       0.23|
|   4332951|        53|      68|     43|     46|       22|     0.68|       0.32|
|  12140731|       133|     

### 3. Из полученного датасета нужно будет выбрать города с
- максимальным и минимальным средним возрастом
- максимальной долей мужчин
- максимальной долей женщин

In [22]:
ok_dem_cities_stat_df = (
    ok_dem_df
    .select(
        sf.max(ok_dem_df.age_avg).alias("max_age_avg"),
        sf.min(ok_dem_df.age_avg).alias("min_age_avg"),
        sf.max(ok_dem_df.men_share).alias("max_men_share"),
        sf.max(ok_dem_df.women_share).alias("max_women_share")
    )
    .first()
)

# Фильтрация городов с максимальным и минимальным средним возрастом
max_age_avg_cities = ok_dem_df.where(ok_dem_df.age_avg == ok_dem_cities_stat_df.max_age_avg)
min_age_avg_cities = ok_dem_df.where(ok_dem_df.age_avg == ok_dem_cities_stat_df.min_age_avg)

# Фильтрация городов с максимальной долей мужчин и максимальной долей женщин
max_men_share_cities = ok_dem_df.where(ok_dem_df.men_share == ok_dem_cities_stat_df.max_men_share)
max_women_share_cities = ok_dem_df.where(ok_dem_df.women_share == ok_dem_cities_stat_df.max_women_share)

# Объединение всех фильтраций в один датафрейм
all_filtered_cities = (
    max_age_avg_cities
    .unionAll(min_age_avg_cities)
    .unionAll(max_men_share_cities)
    .unionAll(max_women_share_cities)
    .distinct()
)

print(ok_dem_cities_stat_df)
all_filtered_cities.show()

Row(max_age_avg=56, min_age_avg=31, max_men_share=1.0, max_women_share=0.7)


+----------+----------+--------+-------+-------+---------+---------+-----------+
|ok_city_id|rs_city_id|user_cnt|age_avg|men_cnt|women_cnt|men_share|women_share|
+----------+----------+--------+-------+-------+---------+---------+-----------+
|  12141018|       105|      86|     56|     68|       18|     0.79|       0.21|
|  12141277|       112|     195|     31|    106|       89|     0.54|       0.46|
|   3924702|       153|      19|     47|     19|        0|      1.0|        0.0|
|   1593862|        88|      37|     45|     11|       26|      0.3|        0.7|
+----------+----------+--------+-------+-------+---------+---------+-----------+



### 4. Для этих городов в данных Росстат нужно будет выбрать:
#### Файл product_stat
- city_name
- cheapest_product_name самый дешевый товар
- most_expensive_product_name самый дорогой товар
- price_difference разницу в цене между самым дорогим и самым дешевым

In [23]:
# Статистика цен на товары по городам
city_product_stat_df = (
    all_filtered_cities
    .join(prices_df, prices_df.city_id == all_filtered_cities.rs_city_id, how='inner')
    .join(products_for_stat_df, "product_id", how="inner")

    .groupBy(all_filtered_cities.ok_city_id, all_filtered_cities.rs_city_id)
    .agg(
        sf.min(sf.col("price")).alias("cheapest_price"),
        sf.max(sf.col("price")).alias("most_expensive_price")
    )
)

# Статистика цен на товары по городам
cheapest_product_df = (
    all_filtered_cities
    .join(prices_df, prices_df.city_id == all_filtered_cities.rs_city_id, how='inner')
    .join(products_for_stat_df, "product_id", how="inner")

    .join(products_df, "product_id", how="inner")
    .join(city_product_stat_df, "rs_city_id")
    
    .where(sf.col("price") == sf.col("cheapest_price"))
    .withColumnRenamed("product", "cheapest_product_name")
    .select(
        sf.col("rs_city_id"),
        sf.col("cheapest_price"),
        sf.col("cheapest_product_name")
    )
)

most_expensive_product_df = (
    all_filtered_cities
    .join(prices_df, prices_df.city_id == all_filtered_cities.rs_city_id, how='inner')
    .join(products_for_stat_df, "product_id", how="inner")
    
    .join(products_df, "product_id", how="inner")
    .join(city_product_stat_df, "rs_city_id")
    
    .where(sf.col("price") == sf.col("most_expensive_price"))
    .withColumnRenamed("product", "most_expensive_product_name")
    .select(
        sf.col("rs_city_id"),
        sf.col("most_expensive_price"),
        sf.col("most_expensive_product_name")
    )
)

result_df = (
    cheapest_product_df
    .join(most_expensive_product_df, "rs_city_id", how='inner')
    .join(cities_df, sf.col("city_id") == sf.col("rs_city_id"))
    .withColumn(
        "price_difference",
        sf.round(most_expensive_product_df.most_expensive_price - cheapest_product_df.cheapest_price, 2)
    )
    .select(
        sf.col("city").alias("city_name"),
        sf.col("cheapest_product_name"),
        sf.col("most_expensive_product_name"),
        sf.col("price_difference")
    )
)

show_schema(city_product_stat_df)
show_schema(cheapest_product_df)
show_schema(most_expensive_product_df)
show_schema(result_df)

root
 |-- ok_city_id: integer (nullable = true)
 |-- rs_city_id: integer (nullable = true)
 |-- cheapest_price: float (nullable = true)
 |-- most_expensive_price: float (nullable = true)



Total rows: 4
+----------+----------+--------------+--------------------+
|ok_city_id|rs_city_id|cheapest_price|most_expensive_price|
+----------+----------+--------------+--------------------+
|  12141277|       112|           0.6|            32120.32|
|   1593862|        88|          0.62|           619286.56|
|   3924702|       153|           0.7|            909922.5|
+----------+----------+--------------+--------------------+
only showing top 3 rows

root
 |-- rs_city_id: integer (nullable = true)
 |-- cheapest_price: float (nullable = true)
 |-- cheapest_product_name: string (nullable = true)



Total rows: 4


+----------+--------------+---------------------+
|rs_city_id|cheapest_price|cheapest_product_name|
+----------+--------------+---------------------+
|        88|          0.62| Предоставление ме...|
|       105|          1.19|      Спички, коробок|
|       153|           0.7| Предоставление ме...|
+----------+--------------+---------------------+
only showing top 3 rows

root
 |-- rs_city_id: integer (nullable = true)
 |-- most_expensive_price: float (nullable = true)
 |-- most_expensive_product_name: string (nullable = true)



Total rows: 4


+----------+--------------------+---------------------------+
|rs_city_id|most_expensive_price|most_expensive_product_name|
+----------+--------------------+---------------------------+
|        88|           619286.56|       Легковой автомоби...|
|       112|            32120.32|       Холодильник двухк...|
|       105|            790490.5|       Легковой автомоби...|
+----------+--------------------+---------------------------+
only showing top 3 rows

root
 |-- city_name: string (nullable = true)
 |-- cheapest_product_name: string (nullable = true)
 |-- most_expensive_product_name: string (nullable = true)
 |-- price_difference: float (nullable = true)



Total rows: 4
+------------+---------------------+---------------------------+----------------+
|   city_name|cheapest_product_name|most_expensive_product_name|price_difference|
+------------+---------------------+---------------------------+----------------+
|      Тихвин| Предоставление ме...|       Легковой автомоби...|       619285.94|
| Симферополь|      Спички, коробок|       Легковой автомоби...|        790489.3|
|Новороссийск| Предоставление ме...|       Холодильник двухк...|        32119.72|
+------------+---------------------+---------------------------+----------------+
only showing top 3 rows



# Сохранение результата на hdfs

In [24]:
# Сохранение результата на hdfs
def save_to_hdfs():
    (price_stat_df
     .repartition(1)
     .sortWithinPartitions(sf.col("max_price").desc())
     .write
     .mode("overwrite")
     .option("header", "true")
     .option("sep", ";")
     .csv(f"{output_path}/price_stat")
    )
    
    (ok_dem_df
     .repartition(1)
     .sortWithinPartitions(sf.col("user_cnt").desc())
     .write
     .mode("overwrite")
     .option("header", "true")
     .option("sep", ";")
     .csv(f"{output_path}/ok_dem")
    )
    
    (result_df
     .repartition(1)
     .sortWithinPartitions(sf.col("price_difference").desc())
     .write
     .mode("overwrite")
     .option("header", "true")
     .option("sep", ";")
     .csv(f"{output_path}/product_stat")
    )
    
    # Stage
    (all_filtered_cities
     .repartition(1)
     .sortWithinPartitions(sf.col("user_cnt").desc())
     .write
     .mode("overwrite")
     .option("header", "true")
     .option("sep", ";")
     .csv(f"{output_path}/stage/all_filtered_cities_v2")
    )
    
    (cheapest_product_df
     .repartition(1)
     .sortWithinPartitions(sf.col("cheapest_price").desc())
     .write
     .mode("overwrite")
     .option("header", "true")
     .option("sep", ";")
     .csv(f"{output_path}/stage/most_expensive_product_df")
    )
    
    (most_expensive_product_df
     .repartition(1)
     .sortWithinPartitions(sf.col("most_expensive_price").desc())
     .write
     .mode("overwrite")
     .option("header", "true")
     .option("sep", ";")
     .csv(f"{output_path}/stage/most_expensive_product_df")
    )

In [25]:
save_to_hdfs()

In [26]:
!echo "------Output dir-----"
!hdfs dfs -ls {output_path}
!echo "------Stage dir-----"
!hdfs dfs -ls {output_path}/stage

------Output dir-----
Found 4 items
drwxr-xr-x   - t.starzhevskij hadoop          0 2024-04-20 18:28 /user/t.starzhevskij/task4/ok_dem
drwxr-xr-x   - t.starzhevskij hadoop          0 2024-04-20 18:28 /user/t.starzhevskij/task4/price_stat
drwxr-xr-x   - t.starzhevskij hadoop          0 2024-04-20 18:28 /user/t.starzhevskij/task4/product_stat
drwxr-xr-x   - t.starzhevskij hadoop          0 2024-04-20 18:28 /user/t.starzhevskij/task4/stage
------Stage dir-----
Found 3 items
drwxr-xr-x   - t.starzhevskij hadoop          0 2024-04-20 18:28 /user/t.starzhevskij/task4/stage/all_filtered_cities_v2
drwxr-xr-x   - t.starzhevskij hadoop          0 2024-04-16 09:49 /user/t.starzhevskij/task4/stage/filtered_cities
drwxr-xr-x   - t.starzhevskij hadoop          0 2024-04-20 18:28 /user/t.starzhevskij/task4/stage/most_expensive_product_df


# Если собрать все в одном месте

In [52]:
from pyspark.sql.types import *
import pyspark.sql.functions as sf
from pyspark.sql import types

# Read
cities_df = (
    spark.read
        .option("header", "false")
        .option("sep", ";")
        .csv(city_path)
)

products_df = (
    spark.read
        .option("header", "false")
        .option("sep", ";")
        .csv(product_path)
)

prices_df = (
    spark.read
        .option("header", "false")
        .option("sep", ";")
        .csv(prices_path)
)

products_for_stat_df = (
    spark.read
        .option("header", "false")
        .option("sep", ";")
        .csv(products_for_stat_path)
)

# (user_id, create_date, birth_date (число дней с 1970-01-01), gender, id_country, id_city, login_region)
users_df = (
    spark.read
        .option("header", "false")
        .option("sep", "\t")
        .csv(demography_path)
)

# Справочник стран geography/countries.csv (name,id)
countries_df = (
    spark.read
        .option("header", "false")
        .option("sep", ",")
        .csv(country_path)
)

rs_city_df = (
    spark.read
        .option("header", "false")
        .option("sep", "\t")
        .csv(rs_city_path)
)

## Приводим типы и задаем названия полей.
cities_df = (
    cities_df
    .select(
        sf.col("_c0").alias("city"),
        sf.col("_c1").cast(IntegerType()).alias("city_id")
    )
)

products_df = (
    products_df
    .select(
        sf.col("_c0").alias("product"),
        sf.col("_c1").cast(IntegerType()).alias("product_id")
    )
)

prices_df = (
    prices_df
    .select(
        sf.col("_c0").cast(IntegerType()).alias("city_id"),
        sf.col("_c1").cast(IntegerType()).alias("product_id"),
        # sf.replace(sf.col("_c2"), ",", ".").cast(DoubleType()).alias("price")
        sf.regexp_replace(
                    sf.col("_c2"), 
                    ",",
                    ".").cast(types.FloatType()).alias("price")
    )
)

products_for_stat_df = (
    products_for_stat_df
    .select(
        sf.col("_c0").cast(IntegerType()).alias("product_id")
    )
)

rs_city_df = (
    rs_city_df
    .select(
        sf.col("_c0").cast(types.IntegerType()).alias("ok_city_id"),
        sf.col("_c1").cast(types.IntegerType()).alias("rs_city_id"),
    )
)

users_df = (
    users_df
    .select(
        sf.col("_c0").cast(types.IntegerType()).alias("user_id"),
        sf.col("_c1").cast(types.LongType()).alias("create_date"),
        sf.from_unixtime(sf.col("_c2") * (24 * 60 * 60)).alias('birth_date').cast(types.DateType()),
        sf.col("_c2").cast(types.LongType()).alias("birth_date_days"),
        sf.col("_c3").cast(types.ShortType()).alias("gender"),
        sf.col("_c4").cast(types.LongType()).alias("country_id"),
        sf.col("_c5").cast(types.IntegerType()).alias("city_id"),
        sf.col("_c6").cast(types.IntegerType()).alias("login_region")
    )
)

# Queries

## Статистика цен на товары по городам
price_stat_df = (
    products_for_stat_df
    .join(prices_df, "product_id", how='inner')
    .groupBy(prices_df.product_id)
    .agg(
        sf.round(sf.max(prices_df.price), 2).alias("max_price"),
        sf.round(sf.min(prices_df.price), 2).alias("min_price"),
        sf.round(sf.avg(prices_df.price), 2).alias("avg_price")
    )
    .orderBy(sf.col("max_price").desc())
)

## Города с подсчетом количества продуктов, цены на которые выше средних
citites_for_stat_df = (
    prices_df
    .join(price_stat_df, "product_id")
    .where(prices_df.price > price_stat_df.avg_price)
    .groupBy(prices_df.city_id)
    .agg(
        sf.count(prices_df.city_id).alias("product_cnt")
    )
    .orderBy(sf.col("product_cnt").desc())
    .join(rs_city_df, rs_city_df.rs_city_id == prices_df.city_id)
    .select(
        sf.col("ok_city_id"),
        sf.col("rs_city_id"),
        sf.col("product_cnt")
    )
)

## Пользователи из требуемых городов со статистикой
current_date = sf.lit(current_dt).cast(types.DateType())
age = sf.floor(sf.datediff(current_date, users_df.birth_date) / 365.25).alias('age')

ok_dem_df = (
    users_df
    .join(citites_for_stat_df, citites_for_stat_df.ok_city_id == users_df.city_id)
    .select(
        citites_for_stat_df.ok_city_id,
        citites_for_stat_df.rs_city_id,
        users_df.user_id,
        users_df.gender,
        age,
    )
    
    .groupBy(citites_for_stat_df.ok_city_id, citites_for_stat_df.rs_city_id)
    .agg(
        sf.count(users_df.user_id).alias("user_cnt"),
        sf.floor(sf.avg(sf.col("age"))).alias("age_avg"),
        sf.sum(sf.when(users_df.gender == 2, 1).otherwise(0)).alias("men_cnt"),
        sf.sum(sf.when(users_df.gender == 1, 1).otherwise(0)).alias("women_cnt")
    )
    .withColumn("men_share", sf.round(sf.col("men_cnt") / sf.col('user_cnt'), 2))
    .withColumn("women_share", sf.round(sf.col("women_cnt") / sf.col('user_cnt'), 2))
)

## Города, с фильтрацией условий для пользователей
ok_dem_cities_stat_df = (
    ok_dem_df
    .select(
        sf.max(ok_dem_df.age_avg).alias("max_age_avg"),
        sf.min(ok_dem_df.age_avg).alias("min_age_avg"),
        sf.max(ok_dem_df.men_share).alias("max_men_share"),
        sf.max(ok_dem_df.women_share).alias("max_women_share")
    )
    .first()
)

# Фильтрация городов с максимальным и минимальным средним возрастом
max_age_avg_cities = ok_dem_df.where(ok_dem_df.age_avg == ok_dem_cities_stat_df.max_age_avg)
min_age_avg_cities = ok_dem_df.where(ok_dem_df.age_avg == ok_dem_cities_stat_df.min_age_avg)

# Фильтрация городов с максимальной долей мужчин и максимальной долей женщин
max_men_share_cities = ok_dem_df.where(ok_dem_df.men_share == ok_dem_cities_stat_df.max_men_share)
max_women_share_cities = ok_dem_df.where(ok_dem_df.women_share == ok_dem_cities_stat_df.max_women_share)

# Объединение всех фильтраций в один датафрейм
all_filtered_cities = (
    max_age_avg_cities
    .unionAll(min_age_avg_cities)
    .unionAll(max_men_share_cities)
    .unionAll(max_women_share_cities)
    .distinct()
)

## Статистика цен на товары по требуемым городам
city_product_stat_df = (
    all_filtered_cities
    .join(prices_df, prices_df.city_id == all_filtered_cities.rs_city_id, how='inner')
    .join(products_for_stat_df, "product_id", how="inner")

    .groupBy(all_filtered_cities.ok_city_id, all_filtered_cities.rs_city_id)
    .agg(
        sf.min(sf.col("price")).alias("cheapest_price"),
        sf.max(sf.col("price")).alias("most_expensive_price")
    )
)

cheapest_product_df = (
    all_filtered_cities
    .join(prices_df, prices_df.city_id == all_filtered_cities.rs_city_id, how='inner')
    .join(products_for_stat_df, "product_id", how="inner")

    .join(products_df, "product_id", how="inner")
    .join(city_product_stat_df, "rs_city_id")
    
    .where(sf.col("price") == sf.col("cheapest_price"))
    .withColumnRenamed("product", "cheapest_product_name")
    .select(
        sf.col("rs_city_id"),
        sf.col("cheapest_price"),
        sf.col("cheapest_product_name")
    )
)

most_expensive_product_df = (
    all_filtered_cities
    .join(prices_df, prices_df.city_id == all_filtered_cities.rs_city_id, how='inner')
    .join(products_for_stat_df, "product_id", how="inner")
    
    .join(products_df, "product_id", how="inner")
    .join(city_product_stat_df, "rs_city_id")
    
    .where(sf.col("price") == sf.col("most_expensive_price"))
    .withColumnRenamed("product", "most_expensive_product_name")
    .select(
        sf.col("rs_city_id"),
        sf.col("most_expensive_price"),
        sf.col("most_expensive_product_name")
    )
)

result_df = (
    cheapest_product_df
    .join(most_expensive_product_df, "rs_city_id", how='inner')
    .join(cities_df, sf.col("city_id") == sf.col("rs_city_id"))
    .withColumn(
        "price_difference",
        sf.round(most_expensive_product_df.most_expensive_price - cheapest_product_df.cheapest_price, 2)
    )
    .select(
        sf.col("city").alias("city_name"),
        sf.col("cheapest_product_name"),
        sf.col("most_expensive_product_name"),
        sf.col("price_difference")
    )
)

In [54]:
show_schema(result_df)
save_to_hdfs()

root
 |-- city_name: string (nullable = true)
 |-- cheapest_product_name: string (nullable = true)
 |-- most_expensive_product_name: string (nullable = true)
 |-- price_difference: float (nullable = true)

Total rows: 4


+------------+---------------------+---------------------------+----------------+
|   city_name|cheapest_product_name|most_expensive_product_name|price_difference|
+------------+---------------------+---------------------------+----------------+
|      Тихвин| Предоставление ме...|       Легковой автомоби...|       619285.94|
| Симферополь|      Спички, коробок|       Легковой автомоби...|        790489.3|
|Новороссийск| Предоставление ме...|       Холодильник двухк...|        32119.72|
+------------+---------------------+---------------------------+----------------+
only showing top 3 rows



In [55]:
!echo "------Output dir-----"
!hdfs dfs -ls {output_path}
!echo "------Stage dir-----"
!hdfs dfs -ls {output_path}/stage

------Output dir-----
Found 4 items
drwxr-xr-x   - t.starzhevskij hadoop          0 2024-04-20 18:42 /user/t.starzhevskij/task4/ok_dem
drwxr-xr-x   - t.starzhevskij hadoop          0 2024-04-20 18:42 /user/t.starzhevskij/task4/price_stat
drwxr-xr-x   - t.starzhevskij hadoop          0 2024-04-20 18:43 /user/t.starzhevskij/task4/product_stat
drwxr-xr-x   - t.starzhevskij hadoop          0 2024-04-20 18:43 /user/t.starzhevskij/task4/stage
------Stage dir-----
Found 3 items
drwxr-xr-x   - t.starzhevskij hadoop          0 2024-04-20 18:43 /user/t.starzhevskij/task4/stage/all_filtered_cities_v2
drwxr-xr-x   - t.starzhevskij hadoop          0 2024-04-16 09:49 /user/t.starzhevskij/task4/stage/filtered_cities
drwxr-xr-x   - t.starzhevskij hadoop          0 2024-04-20 18:43 /user/t.starzhevskij/task4/stage/most_expensive_product_df


In [56]:
# После работы обязательно отключаем спарк и отдаем ресурсы!
spark.stop()